In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time

In [10]:
pd.read_csv('./data/games.csv')

,link,round,home_team,away_team,result,home_position_all,away_position_all,home_winperc_all,away_winperc_all,home_position_home,...,home_form_4,home_form_5,away_form_1,away_form_2,away_form_3,away_form_4,away_form_5,home_previous,draw_previous,away_previous
0,https://footystats.org//england/leicester-city...,31,Leicester City,Brighton & Hove Albion,Draw,3,15,0.53,0.23,3,...,Draw,Win,Draw,Draw,Draw,Lose,Lose,0.55,0.18,0.27
1,https://footystats.org//england/tottenham-hots...,31,Tottenham Hotspur,West Ham United,Home,8,17,0.37,0.23,5,...,Lose,Win,Lose,Lose,Lose,Lose,Lose,0.54,0.15,0.31
2,https://footystats.org//england/manchester-uni...,31,Manchester United,Sheffield United,Home,5,8,0.40,0.37,5,...,Draw,Lose,Lose,Draw,Win,Win,Win,0.50,0.50,0.00
3,https://footystats.org//england/newcastle-unit...,31,Newcastle United,Aston Villa,Draw,13,19,0.33,0.23,10,...,Draw,Win,Lose,Lose,Lose,Draw,Draw,0.42,0.36,0.21
4,https://footystats.org//england/norwich-city-f...,31,Norwich City,Everton,Away,20,12,0.17,0.33,19,...,Win,Draw,Lose,Lose,Win,Draw,Lose,0.27,0.54,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,https://footystats.org//england/watford-fc-vs-...,1,Watford,Brighton & Hove Albion,Away,17,5,0.00,0.00,18,...,Lose,Win,Draw,Lose,Draw,Lose,Lose,0.38,0.38,0.23
298,https://footystats.org//england/tottenham-hots...,1,Tottenham Hotspur,Aston Villa,Home,16,4,0.00,0.00,17,...,Draw,Lose,Lose,Draw,Win,Win,Win,0.65,0.20,0.15
299,https://footystats.org//england/leicester-city...,1,Leicester City,Wolverhampton Wanderers,Draw,12,15,0.00,0.00,11,...,Win,Win,Win,Win,Win,Lose,Win,0.40,0.20,0.40
300,https://footystats.org//england/newcastle-unit...,1,Newcastle United,Arsenal,Away,14,10,0.00,0.00,14,...,Win,Lose,Draw,Lose,Win,Win,Lose,0.10,0.14,0.76


In [3]:
url='https://footystats.org//england/chelsea-fc-vs-sheffield-united-fc-h2h-stats#579026'
driver = webdriver.Firefox(executable_path='geckodriver.exe')
driver.get(url)
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [7]:
def find_result(home_goals, away_goals):
    if home_goals > away_goals:
        return 'Home'
    elif away_goals > home_goals:
        return 'Away'
    else:
        return 'Draw'


def form_home(result):
    home = int(result[0])
    away = int(result[1])
    if home > away:
        return 'Win'
    elif away > home:
        return 'Lose'
    else:
        return 'Draw'
    
def form_away(result):
    home = int(result[0])
    away = int(result[1])
    if away > home:
        return 'Win'
    elif home > away:
        return 'Lose'
    else:
        return 'Draw'

In [9]:
tables = pd.read_html(html)
teams = soup.find_all('h2', {'class': 'mt01e ac bold'})
final = {}
final['link'] = url
final['round'] = int(soup.find('p', {'class': 'mt05e'}).find('span').text)
final['home_team'] = teams[0].text
final['away_team'] = teams[1].text
goals = soup.find('div', {'class', 'widget-content'}).find('h2').text.split('-')
home_goals = int(goals[0].strip())
away_goals = int(goals[1].strip())
final['result'] = find_result(home_goals, away_goals)
class_all = tables[6]
final['home_position_all'] = class_all.loc[class_all['Team'] == final['home_team'], '#'].values[0]
final['away_position_all'] = class_all.loc[class_all['Team'] == final['away_team'], '#'].values[0]
final['home_winperc_all'] = float(class_all.loc[class_all['Team'] == final['home_team'], 'Win%'].values[0].strip('%')) / 100
final['away_winperc_all'] = float(class_all.loc[class_all['Team'] == final['away_team'], 'Win%'].values[0].strip('%')) / 100
class_home = tables[4]
final['home_position_home'] = int(class_home.loc[class_home['Team'] == final['home_team'], '#'].values[0])
final['away_position_home'] = int(class_home.loc[class_home['Team'] == final['away_team'], '#'].values[0])
final['home_winperc_home'] = float(
    class_home.loc[class_home['Team'] == final['home_team'], 'Win%'].values[0].strip('%')) / 100
final['away_winperc_home'] = float(
    class_home.loc[class_home['Team'] == final['away_team'], 'Win%'].values[0].strip('%')) / 100
class_away = tables[5]
final['home_position_away'] = class_away.loc[class_away['Team'] == final['home_team'], '#'].values[0]
final['away_position_away'] = class_away.loc[class_away['Team'] == final['away_team'], '#'].values[0]
final['home_winperc_away'] = float(
    class_away.loc[class_away['Team'] == final['home_team'], 'Win%'].values[0].strip('%')) / 100
final['away_winperc_away'] = float(
    class_away.loc[class_away['Team'] == final['away_team'], 'Win%'].values[0].strip('%')) / 100
home_form_results = soup.find_all('div', {'class': 'h2h-history-results-wrapper'})[0].find_all('div', {
    'class': 'w16 fl ac score-box'})
home_form = [form_home(list(map(str.strip, item.text.split('-')))) for item in home_form_results]
final['home_form_1'] = home_form[0]
final['home_form_2'] = home_form[1]
final['home_form_3'] = home_form[2]
final['home_form_4'] = home_form[3]
final['home_form_5'] = home_form[4]
away_form_results = soup.find_all('div', {'class': 'h2h-history-results-wrapper'})[1].find_all('div', {
    'class': 'w16 fl ac score-box'})
away_form = [form_away(list(map(str.strip, item.text.split('-')))) for item in away_form_results]
final['away_form_1'] = away_form[0]
final['away_form_2'] = away_form[1]
final['away_form_3'] = away_form[2]
final['away_form_4'] = away_form[3]
final['away_form_5'] = away_form[4]
final['home_previous'] = float(soup.find('div', {'class': 'w30 fl ac lh14e teamA pr r-w20'}).find('span', {'class': 'dark-gray'}).text.replace('(', '').replace('%)', '')) / 100
try:
    final['draw_previous'] = float(
    soup.find('div', {'class': 'lh14e draw-line semi-bold'}).find('span', {'class': 'dark-gray'}).text.replace('(',
                                                                                                               '').replace(
        '%)', '')) / 100
except:
    final['draw_previous'] = 0
final['away_previous'] = float(
    soup.find('div', {'class': 'w30 fl ac lh14e teamB pr r-w20'}).find('span', {'class': 'dark-gray'}).text.replace(
        '(', '').replace('%)', '')) / 100

AttributeError: 'NoneType' object has no attribute 'find'